In [3]:
!pip install diffusers
import torch
from diffusers import StableDiffusionPipeline as sdp
from PIL import ImageTk, Image
import io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.7 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
class CFG:

    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    seed = 42
    generator = torch.Generator(device).manual_seed(seed)
    image_gen_steps = 35
    image_gen_model_id = "stabilityai/stable-diffusion-2"
    image_gen_size = (400, 400)
    image_gen_guidance_scale = 9
    prompt_max_length = 12


In [6]:
image_gen_model = sdp.from_pretrained(
    CFG.image_gen_model_id, torch_dtype=CFG.dtype, variant="fp16" if CFG.device == "cuda" else None
)
image_gen_model = image_gen_model.to(CFG.device)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def generate_image(prompt, model):
    # Generate an image from the given text prompt
    with torch.autocast(CFG.device):
        image = model(
            prompt, num_inference_steps=CFG.image_gen_steps,
            generator=CFG.generator,
            guidance_scale=CFG.image_gen_guidance_scale
        ).images[0]

    image = image.resize(CFG.image_gen_size)
    return image


In [8]:
def display_image(image):
    # Convert the image to a format Tkinter can display
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format="PNG")
    img_byte_arr = img_byte_arr.getvalue()

    img = Image.open(io.BytesIO(img_byte_arr))
    img_tk = ImageTk.PhotoImage(img)

    return img_tk

In [ ]:
prompt = input("Write the Prompt")
image = generate_image(prompt, image_gen_model)
display_image(image)

  0%|          | 0/35 [00:00<?, ?it/s]